# HW4 - Convert WikiHow to PDDL

In this homework we will translate a wikHow article into the Planning Domain Definition Language (PDDL).    The goals of this homework are to:
1. Deepen our understanding of classical planning
2. Understand the process of manually translating from natural language instructions into PDDL.
3. Think about how we might teach GPT-3 to perform the translation automatically (don't worry – you don't have to implement the automatic translation in this homework).
4. Capture some data that might be useful for fine-tuning GPT-3 to convert to PDDL (for possible use in some students' term projects).  

In [98]:
# Install the PDDL parser
!git clone https://github.com/pucrs-automated-planning/pddl-parser
!cd pddl-parser

fatal: destination path 'pddl-parser' already exists and is not an empty directory.


In [99]:
# Mount your Google Drive so that you can save your PDDL files.
from google.colab import drive
drive.mount('/content/drive')
path = '/content/drive/My Drive/CIS 700/HW4'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


These are some helper functions for loading and saving files, and for displaying long texts.

In [100]:
import os
import json
import textwrap


## Create a file from the contents of a string. 
def write_to_file(string, path, filename):
  """This is a helper function to create a file from a string."""
  if not os.path.exists(path) and path != "":
    os.makedirs(path)
  with open(os.path.join(path, filename), "w") as f:
      f.write(string)


# Load a JSON file given a filename
# If the file doesn't exist, then return an empty dictionary instead
def load_json(filename):
    try:
        with open(filename, 'r') as f:
            return json.load(f)
    except FileNotFoundError:
        return {}

# Save a JSON file given a full path filename and a dictionary
# If the path doesn't exist, create the directories before saving the file.
def save_json(filename, data):
    if not os.path.exists(os.path.dirname(filename)):
        os.makedirs(os.path.dirname(filename))
    with open(filename, 'w') as f:
        json.dump(data, f, indent=4)


def wrap_text(text):
  wrapped_text = ""
  lines = text.strip().split("\n")

  for line in lines:
    if len(line) > 90:
        w = textwrap.TextWrapper(width=90, break_long_words=False)
        line = '\n'.join(w.wrap(line))
    wrapped_text += line + "\n"
  return wrapped_text


### Step 1: Pick an Interesting wikiHow Article

We'll create our PDDL files from a wikiHow article.  The goal for this is to start from something that describes proceedures and actions and is written in natural language, and then to manually translate it into the description language used for automated planning.

Here are a few wikiHow articles that I thought might be interesting since they had some elements that could make for compelling interactive fiction.  It's fine to pick your own article.   We won't translate the whole article, just a few steps, so you can pick out the parts that you think are most relevant / easiest to create action schema from.

Survival Stories
* [How to Survive in the Woods](https://www.wikihow.com/Survive-in-the-Woods)
* [How to Survive in the Jungle](https://www.wikihow.com/Survive-in-the-Jungle)
* [How to Survive on a Desert Island](https://www.wikihow.com/Survive-on-a-Desert-Island
) 
* [How to Survive on a Deserted Island With Nothing](https://www.wikihow.com/Survive-on-a-Deserted-Island-With-Nothing)
* [How to Get Out of Quicksand](https://www.wikihow.com/Get-Out-of-Quicksand)
* [How to Open a Coconut](https://www.wikihow.com/Open-a-Coconut)
* [How to Test if a Plant Is Edible](https://www.wikihow.com/Test-if-a-Plant-Is-Edible)
* [How to Find True North Without a Compass](https://www.wikihow.com/Find-True-North-Without-a-Compass)
* [How to Survive a Wolf Attack](https://www.wikihow.com/Survive-a-Wolf-Attack)

Kid Detectives
* [How to Make a Detective Kit](https://www.wikihow.com/Make-a-Detective-Kit)
* [How to Disguise Yourself](https://www.wikihow.com/Disguise-Yourself)
* [How to Make a Hidden Camera](https://www.wikihow.com/Make-a-Hidden-Camera)
* [How to Hide Money](https://www.wikihow.com/Hide-Money)
* [How to Spy on People](https://www.wikihow.com/Spy-on-People)
* [How to Hack](https://www.wikihow.com/Hack)
* [How to Make a Grappling Hook](https://www.wikihow.com/Make-a-Grappling-Hook)
* [How to Open a Locked Door](https://www.wikihow.com/Open-a-Locked-Door)
* [How to Create a Secret Society](https://www.wikihow.com/Create-a-Secret-Society)
* [How to Win Fights at School](https://www.wikihow.com/Win-Fights-at-School)

Dystopian Futures
* [How to Survive a Comet Hitting Earth](https://www.wikihow.com/Survive-a-Comet-Hitting-Earth)
* [How to Survive an EMP](https://www.wikihow.com/Survive-an-EMP)
* [How to Survive a Nuclear Attack](https://www.wikihow.com/Survive-a-Nuclear-Attack)
* [How to Build a Fallout Shelter](https://www.wikihow.com/Build-a-Fallout-Shelter)
* [How to Survive a Riot](https://www.wikihow.com/Survive-a-Riot)
* [How to Survive Under Martial Law](https://www.wikihow.com/Survive-Under-Martial-Law)
* [How to Avoid Danger During Civil Unrest](https://www.wikihow.com/Avoid-Danger-During-Civil-Unrest)
* [How to Thwart an Abduction Attempt](https://www.wikihow.com/Thwart-an-Abduction-Attempt)
* [How to Make Papyrus](https://www.wikihow.com/Make-Papyrus)


As an example, I'll pick the [How to Survive in the Woods](https://www.wikihow.com/Survive-in-the-Woods) article, and work on translating Part 1, Step 1 into PDDL.  Here is step 1 from that article: 


<center>
<img src="https://www.wikihow.com/images/thumb/c/c0/Survive-in-the-Woods-Step-11-Version-5.jpg/aid31352-v4-728px-Survive-in-the-Woods-Step-11-Version-5.jpg.webp" class="img-responsive" alt="Search for a source of fresh water. (Creative Commons License)"/>
</center>



> ### Finding Drinking Water
> Search for a source of fresh water.  The first thing that you'll need in order to survive in the woods is water that you can drink. Look for signs of fresh water nearby like areas of green foliage that indicate water is nearby, low-lying areas where water could be collected, and signs of wildlife like animal tracks. It could mean that a creek, stream, or pond is nearby. While finding water is important for survival, be aware some water sources will not be safe - if possible treat all drinking water before using it. 
If there are mountains nearby, look for water collected at the foot of the cliffs.
> * The presence of insects like mosquitoes and flies means that water is nearby.
> * Water from heavily oxygenated water (such as from a big waterfall or rapids) typically is safer than that from a slow or still water source.
> * Freshwater springs are typically safer water sources, although these can be contaminated by mineral or bacteria as well.
> * Remember that all untreated water must be considered risky unless treated. Even crystal clear water can harbor diseases and be dangerous if consumed.

# Step 2: Manually Create a PDDL


## Domain and Problems

You should create PDDL files for:
1. The domain.  You should have a single `domain.pddl` file which defines the domain, including the types, predicates and action schemata that are relevant to your wikiHow article.
2. One or more problems.  You should create one more problem file that defines a problem, an initial state, and a goal, that can be reached by applying the action schema that you defined.  In some cases, it might make sense to have one problem file for each step in a wikiHow article. 

The name of your domain should be the aricle title.  I have started defining a domain for `survive_in_the_woods`.
```
(define (domain survive_in_the_woods)
   (:requirements :strips :typing)
   ...
```   
I'll create a problem file for Step 1 in the article.  I'll name the problem `collect_water` and I'll define the goal as `(:goal (and (inventory npc water))` where the player (or an AI-controled non-playable character, abbreviated as NPC) needs to add water to their inventory. 

Here's what the start and end of the problem PDDL file would look like:

```
(define (problem collect_water)
   (:domain survive_in_the_woods)
...
   (:goal (and (inventory npc water)))
```


## Types

You should specify what the `types` are in your domain.  They should be the things that are relevant to solving the problem that you're tackling.  It's sometimes also useful to create subtypes. Which you can do like this:
```
   (:types        
       water - item
       player direction location 
   )
``` 
By default all types are subtypes of `object`. The line
```
water - item
```
allows us to define `water` as a subtype of `item` (`item` is also introduced as a type in this same line).   Having `water` as a subtype allows us to restrict some action schemas to only operate on that type.


## Action Schema

Here's an example of an action schema for getting items:
```
   (:action get
      :parameters (?item - item ?p - player ?l1 - location) 
      :precondition (and (at ?p ?l1) (at ?item ?l1))
      :effect (and (inventory ?p ?item) (not (at ?item ?l1)))
   )
```
It has the effect or removing the item from the current location and putting it into the player's inventory.

We might add a seperate action for getting water, since when our player gets some water from a lake, the water shouldn't be removed from that location.  Here's one way to write it:

```
   (:action get_water
      :parameters (?p - player ?loc - location ?water - water) 
      :precondition (and (at ?p ?loc) (has_water_source ?loc))
      :effect (and (inventory ?p ?water) (not (treated ?water)))
   )
```
(We might also consider adding some additional preconditions, like that the player has a container to store their water in). 

## Predicates

In your PDDL domain file, please define your predicates like this:
```
   (:predicates
      (has_water_source ?loc - location) ; this location has a source of fresh water.
      (treated ?water - water) ; True if the water has been decontaimated by boiling it
      ...
   )
```
You should give the type of each predicate's arguments, and a comment about what the predicate means (everything after the `;` is a comment). 

# Example Domain 

Here's an example of how I started the `survive_in_the_woods` domain.

In [101]:
example_domain = """
(define (domain survive_in_the_woods)
   (:requirements :strips :typing)
   (:types
       water - item 
       player direction location
   )

   (:predicates
      (has_water_source ?loc - location) ; this location has a source of fresh water.
      (treated ?water - water) ; True if the water has been decontaimated by boiling it
      (at ?obj - object ?loc - location) ; an object is at a location 
      (inventory ?player ?item) ; an item is in the player's inventory
      (connected ?loc1 - location ?dir - direction ?loc2 - location) ; location 1 is connected to location 2 in the direction
      (blocked ?loc1 - location ?dir - direction ?loc2 - location) ; the connection between location 1 and 2 in currently blocked
   )

   (:action go ; navigate to an adjacent location 
      :parameters (?dir - direction ?p - player ?l1 - location ?l2 - location) 
      :precondition (and (at ?p ?l1) (connected ?l1 ?dir ?l2) (not (blocked ?l1 ?dir ?l2)))
      :effect (and (at ?p ?l2) (not (at ?p ?l1)))
   )

   (:action get ; pick up an item and put it in the inventory
      :parameters (?item - item ?p - player ?l1 - location) 
      :precondition (and (at ?p ?l1) (at ?item ?l1))
      :effect (and (inventory ?p ?item) (not (at ?item ?l1)))
   )

   (:action get_water ; get water from a location that has a water source like a lake.
      :parameters (?p - player ?loc - location ?water - water) 
      :precondition (and (at ?p ?loc) (has_water_source ?loc))
      :effect (and (inventory ?p ?water) (not (treated ?water)))
   )
)
"""

# Write the domain to a PDDL file
domain_filename = "domain-survive_in_the_woods.pddl"
write_to_file(example_domain, path, domain_filename)

# Example problem

Here's how I specified the problem of collecting water from a source like a waterfall.   

I instantiated several objects, and specified an initial state that would allow the goal of `(inventory npc water)` to be reached by this sequence of actions:

```
go west npc camp path
go west npc path cliff
go up npc cliff waterfall
get_water npc waterfall water
```


In [102]:
example_problem = """
(define (problem collect_water)
   (:domain survive_in_the_woods)

   (:objects
      npc - player
      waterfall camp path cliff - location
      in out north south east west up down - direction
      water - water
   )

   (:init
      (connected camp west path)
      (connected path east camp)
      (connected path west cliff)
      (connected cliff east path)
      (connected cliff up waterfall)
      (connected waterfall down cliff)
      (at npc camp)
      (at canteen camp)
      (has_water_source waterfall)
   )

   (:goal (and (inventory npc water)))
)
"""

# Write the problem to a PDDL file
problem_filename = "problem-collect_water.pddl"
write_to_file(example_problem, path, problem_filename)

I parse my files to check for formatting errors

In [103]:
!python -B pddl-parser/PDDL.py '{path}/{domain_filename}' '{path}/{problem_1_filename}'

----------------------------
['define',
 ['domain', 'survive_in_the_woods'],
 [':requirements', ':strips', ':typing'],
 [':types', 'water', '-', 'item', 'player', 'direction', 'location'],
 [':predicates',
  ['has_water_source', '?loc', '-', 'location'],
  ['treated', '?water', '-', 'water'],
  ['at', '?obj', '-', 'object', '?loc', '-', 'location'],
  ['inventory', '?player', '?item'],
  ['connected',
   '?loc1',
   '-',
   'location',
   '?dir',
   '-',
   'direction',
   '?loc2',
   '-',
   'location'],
  ['blocked',
   '?loc1',
   '-',
   'location',
   '?dir',
   '-',
   'direction',
   '?loc2',
   '-',
   'location']],
 [':action',
  'go',
  ':parameters',
  ['?dir',
   '-',
   'direction',
   '?p',
   '-',
   'player',
   '?l1',
   '-',
   'location',
   '?l2',
   '-',
   'location'],
  ':precondition',
  ['and',
   ['at', '?p', '?l1'],
   ['connected', '?l1', '?dir', '?l2'],
   ['not', ['blocked', '?l1', '?dir', '?l2']]],
  ':effect',
  ['and', ['at', '?p', '?l2'], ['not', ['at'

Finally, I make sure that I can compute a plan

In [104]:
!python -B pddl-parser/planner.py '{path}/{domain_filename}' '{path}/{problem_1_filename}'

Time: 0.006136178970336914s
plan:
go west npc camp path
go west npc path cliff
go up npc cliff waterfall
get_water npc waterfall water



# Other problems

If the player wants to survive in the woods, other problems remain.  For starters, their water still isn't safe to drink!  To fix that problem we could implement Step 6 of [How to Survive in the Woods](https://www.wikihow.com/Survive-in-the-Woods): 


<center>
<img src="https://www.wikihow.com/images/thumb/7/72/Survive-in-the-Woods-Step-6-Version-5.jpg/aid31352-v4-728px-Survive-in-the-Woods-Step-6-Version-5.jpg" class="img-responsive" alt="Purify Water. (Image Copyright wikiHow, Inc)"/>
</center>


> ### Purify any water that you find. 
> It's extremely important that you purify any water that you collect, including rainwater, dew, and ice or snow, so you don't consume bacteria that could make you ill or even kill you. Use a piece of cloth or clothing to strain the water to remove large particles, then boil the water for 10 minutes to kill any contaminants.
> * If you don't have a container to boil water in, you can fill a clear plastic bottle with water, seal the lid, and place the bottle on its side in direct sunlight for 6 hours to purify it.
> * In the event that you have no containers and no way to purify water, you can dig a deep hole, let it fill with groundwater, and wait for the particles to settle at the bottom and the water is clear before you drink it. You should only do this if you have no other option.

I won't give the details of how to implement this, but you can imagine extending our `survive_in_the_woods` domain to add action schema for `strain`, `boil`, and `purify_in_sunlight`, and to create a new PDDL problem for `purify_water` with the goal `(:goal (and (inventory npc water) (treated water)))`.  

That in turn might require us to solve a problem like `create_a_fire` in order to boil the water.

## Step 3: Create JSON file 

Next, you will create a JSON file that keeps information about the article that you created.  Specifically, we'll use this to help specify a mapping between elements of your PDDL and the language of the original wikiHow article that you based it on.  

If anyone is interested in doing a term project focused on automatically converting wikiHow to PDDL, then we'll share this JSON data with your classmates.


In [107]:
domain = "survive_in_the_woods"
json_filename = os.path.join(path, domain + ".json")

data = load_json(json_filename)

data[domain] = {}
data[domain]['wikihow_url'] = 'https://www.wikihow.com/Survive-in-the-Woods'
data[domain]['domain_pddl'] = example_domain

# List your domain's types, predicate and actions.
data[domain]['types'] = ['water', 'item', 'player', 'direction', 'location']
data[domain]['predicates'] = ['has_water_source', 'treated', 'at', 'inventory', 'connected', 'blocked']
data[domain]['actions'] = ['go', 'get', 'get_water']

# Specify the problems that you tackled
data[domain]['problems'] = {}
data[domain]['problems']['collect_water'] = {}
data[domain]['problems']['collect_water']['pddl'] = example_problem

# Give the corresponding step of the wikihow article that you dreived it from
data[domain]['problems']['collect_water']['wikihow_section'] = "Part 1, Step 1"
# Copy and paste the step's text.  You can format it in plain text or in Markdown.
data[domain]['problems']['collect_water']['wikihow_text'] = """
### Finding Drinking Water
Search for a source of fresh water.  The first thing that you'll need in order to survive in the woods is water that you can drink. Look for signs of fresh water nearby like areas of green foliage that indicate water is nearby, low-lying areas where water could be collected, and signs of wildlife like animal tracks. It could mean that a creek, stream, or pond is nearby. While finding water is important for survival, be aware some water sources will not be safe - if possible treat all drinking water before using it. 
If there are mountains nearby, look for water collected at the foot of the cliffs.
* The presence of insects like mosquitoes and flies means that water is nearby.
* Water from heavily oxygenated water (such as from a big waterfall or rapids) typically is safer than that from a slow or still water source.
* Freshwater springs are typically safer water sources, although these can be contaminated by mineral or bacteria as well.
* Remember that all untreated water must be considered risky unless treated. Even crystal clear water can harbor diseases and be dangerous if consumed.
"""


# Save the json file
save_json(json_filename, data)


In [109]:


def annotate_mentions(json_filename):
  """
  Walk through the json file and have the user copy-and-paste mentions for the 
  PDDL elements that are mentioned in the text of the wikihow article.  This
  is useful for later projects that focus on automatically deriving PDDL from 
  text. 
  """
  data = load_json(json_filename)
  for domain in data:
    for problem in data[domain]['problems']:

      print("Problem:", problem.upper(), '\n')
      wrapped_text = wrap_text(data[domain]['problems'][problem]['wikihow_text'])
      print(wrapped_text)
      print('----\n')

      print("Please copy-and-paste phrases from the step that mention types, actions, and predicates from the PDDL.")
      print("A mention can be indirect. If there is more than one mention describing the same thing, then put one per line.")
      print("Enter a blank line when you're done.")

      for pddl_element in ['types', 'predicates', 'actions']:
        if not 'mentions' in data[domain]['problems'][problem]:
          data[domain]['problems'][problem]['mentions'] = {}
        if not pddl_element in data[domain]['problems'][problem]['mentions']:
          data[domain]['problems'][problem]['mentions'][pddl_element] = {}
        for item in data[domain][pddl_element]:
          if not item in data[domain]['problems'][problem]['mentions'][pddl_element]:
            data[domain]['problems'][problem]['mentions'][pddl_element][item] = {}
          else:
            #skip asking for annotation if we have already annotated this one.
            continue
          metions = data[domain]['problems'][problem]['mentions'][pddl_element][item]

          print("Mentions related to the", pddl_element[:-1], item.upper())

          while True:
            mention = input()
            if mention == "":
              break
            mentions.append(mention)
            data[domain]['problems'][problem]['mentions'][pddl_element][item] = mentions
            # Save the json file
            save_json(json_filename, data)
        save_json(json_filename, data)
    print("\nYou're done annotating!".upper())

annotate_mentions(json_filename)

Problem: COLLECT_WATER 

### Finding Drinking Water
Search for a source of fresh water.  The first thing that you'll need in order to survive
in the woods is water that you can drink. Look for signs of fresh water nearby like areas
of green foliage that indicate water is nearby, low-lying areas where water could be
collected, and signs of wildlife like animal tracks. It could mean that a creek, stream,
or pond is nearby. While finding water is important for survival, be aware some water
sources will not be safe - if possible treat all drinking water before using it.
If there are mountains nearby, look for water collected at the foot of the cliffs.
* The presence of insects like mosquitoes and flies means that water is nearby.
* Water from heavily oxygenated water (such as from a big waterfall or rapids) typically
is safer than that from a slow or still water source.
* Freshwater springs are typically safer water sources, although these can be contaminated
by mineral or bacteria as well

KeyboardInterrupt: ignored

In [ ]:
print(json.dumps(data, indent=4, sort_keys=True))